In [ ]:
import bs4
import pandas as pd
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver

# 드라이버 위치 설정
driver_loc = "../chromedriver.exe"

# 드라이버 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080") # 창 크기 선택

# 리눅스처럼 웹 화면 없는 경우에도 실행 가능.
# options.add_argument('headless')                           
# options.add_argument('disable-gpu')
# options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")

# 웹 드라이버 정의
driver = webdriver.Chrome(driver_loc, options=options)

# 창크기 최대화
driver.maximize_window()

# 해당 아이템의 컬럼값 및 컬럼명을 넣을 리스트 생성
rowList = []
titleList = []

# time.sleep할 시간 설정
sleepTime = 5

for i in range(999):
    try:
        # 상품이 포함된 페이지별 이동
        url = 'http://www.enuri.com/list.jsp?cate=03181105&tabType=1&page=' + str(i + 1)
        driver.get(url)
        time.sleep(sleepTime)
            
        # 반복문을 위한 한페이지당 상품의 수 설정
        data = driver.page_source
        bs = bs4.BeautifulSoup(data, 'html.parser')
        ulTag = bs.find('ul', {'id':'listBodyDiv'})
        print(len(ulTag))
        liTags = ulTag.findAll('li', {'class':'prodItem wide'})
        
        # 반복문을 통해 상품의 스펙 정보를 긁어옴
        for j in range(len(liTags)):
            # 스마트워치 상품이 아닌 상품 제외
            if not ((i == 0 and (j == 77 or j == 88))):
                # 상품 페이지로 이동하기 위해 각 상품별 xpath 설정
                itemId = liTags[j].attrs['id']
                itemXpath = '//*[@id="{}"]/div[1]/strong/a'.format(itemId)

                # 각 상품 페이지로 이동
                itemMenu = driver.find_element_by_xpath(itemXpath)
                itemMenu.click()

                # 창이동
                driver.switch_to.window(driver.window_handles[-1])

                # 상품 페이지에서 html 소스 긁어오기
                data = driver.page_source
                bs = bs4.BeautifulSoup(data, 'html.parser')

                # 최저가, 최저가 타이틀(컬럼명) 정보가 포함된 상위태그 추출
                divTag = bs.find("div", attrs={"class":"malllowprice"})
                div2Tag = divTag.find("div", {"class":"malllow__cont"})
                tableTag = div2Tag.find('table', {'class':'malllow__list'})
                tbodyTag = tableTag.find('tbody', {'id':'vip_malllow_item_list_id'})
                trTag = tbodyTag.find('tr')
                tdTag = trTag.find('td', {'class':'malllow__item price'})

                # 상품 스펙 정보를 넣을 리스트 생성
                columnList = []
                
                # 최저가 삽입
                lowPrice = tdTag.find('strong').text.replace(',','')
                columnList.append(lowPrice)
                
                # 중복된 컬럼명이 들어가지 않도록 조건문 설정
                if i == 0 and j == 0:
                    # 최저가 컬럼명 삽입
                    lowPriceTitle = tdTag.find('span').text
                    titleList.append(lowPriceTitle)

                # 반복문을 위한 상품 태그 정보 추출
                div3Tag = bs.find('div', {'id':'enuri_spec_table'})
                table2Tag = div3Tag.findAll('table', {'class':'offerer__table'})
                
                # 반복문을 통해 상품 페이지에서 테이블 태그로 존재하는 상품 스펙을 긁어옴.
                for k in range(len(table2Tag)):
                    tbody2Tag = table2Tag[k].find('tbody')
                    thTags = tbody2Tag.findAll('th', {'scope':'row'})
                    tdTags = tbody2Tag.findAll('td')
                    for l in range(len(tdTags)):
                        # 컬럼명을 추출할 thTags 조건 설정
                        if thTags[l].text != '':
                            # 중복된 컬럼명이 들어가지 않도록 조건문 설정
                            if i == 0 and j == 0:
                                titleList.append(thTags[l].text.strip())
                            # 스펙정보 삽입
                            columnList.append(tdTags[l].text.strip().replace('\t',''))
                            
                print(columnList)
                # 한 상품의 모든 스펙정보를 한 행에 삽입
                rowList.append(columnList)
                print(len(rowList))
                
                # 포커스가 맞춰진 창을 닫고 기존의 창으로 이동
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
    except:
        break